In [ ]:
#visualize model
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.initializers import RandomNormal
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score

In [ ]:
x, y = make_circles(n_samples=1000, factor=.5, noise=0.1)
plt.figure(figsize=(8,6))
plt.scatter(x[:,0], x[:,1], c=y)
plt.show()

In [ ]:
# Illustrate weights across epochs
class WeightCapture(Callback):
    "Capture the weights of each layer of the model"
    def __init__(self, model):
        super().__init__()
        self.mymodel = model
        self.weights = []
        self.epochs = []

    def on_epoch_end(self, epoch, logs=None):
        self.epochs.append(epoch) # remember the epoch axis
        weight = {}
        for layer in self.mymodel.layers:
            if not layer.weights:
                continue
            name = layer._name#.split("/")[0]
            #print('name is__', layer.weights[0])
            weight[name] = layer.weights[0].numpy()
        self.weights.append(weight)
        
def make_mlp(activation, initializer, name):
    "Create a model with specified activation and initalizer"
    model = Sequential([
        Input(shape=(2,), name=name+"0"),
        Dense(5, activation=activation, kernel_initializer=initializer),
        Dense(5, activation=activation, kernel_initializer=initializer),
        Dense(5, activation=activation, kernel_initializer=initializer),
        Dense(5, activation=activation, kernel_initializer=initializer),
        Dense(1, activation="sigmoid", kernel_initializer=initializer)
    ])
    
    for i, layer in enumerate(model.layers):
        layer._name = name+'_'+str(i)
    
    return model

def plotweight(capture_cb):
    "Plot the weights' mean and s.d. across epochs"
    fig, ax = plt.subplots(2, 1, sharex=True, constrained_layout=True, figsize=(8, 10))
    ax[0].set_title("Mean weight")
    for key in capture_cb.weights[0]:
        ax[0].plot(capture_cb.epochs, [w[key].mean() for w in capture_cb.weights], label=key)
    ax[0].legend()
    ax[1].set_title("S.D.")
    for key in capture_cb.weights[0]:
        ax[1].plot(capture_cb.epochs, [w[key].std() for w in capture_cb.weights], label=key)
    ax[1].legend()
    plt.show()

In [ ]:
n_epochs=100
batch_size=32 # =dataset size of GD, 1 for SGD, some n for minibatchGD
initializer = RandomNormal(mean=0, stddev=1)

model = make_mlp("sigmoid", initializer, name="sigmoid")

capture_cb = WeightCapture(model)
capture_cb.on_epoch_end(-1)

model.summary()
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])
model.summary()

print("Before training: Accuracy", accuracy_score(y, (model(x).numpy() > 0.5).astype(int)))

model.fit(x, y, batch_size=batch_size, epochs=n_epochs, callbacks=[capture_cb], verbose=0)

print("After training: Accuracy", accuracy_score(y, (model(x).numpy() > 0.5).astype(int)))
print(model.evaluate(x,y))

plotweight(capture_cb)